[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/04-langchain-chat.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/04-langchain-chat.ipynb)

In [1]:
!pip install -qU langchain openai


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


We'll start by initializing the `ChatOpenAI` object. For this we'll need an [OpenAI API key](https://platform.openai.com/account/api-keys). Note that there is naturally a small cost to running this notebook due to the paid nature of OpenAI's API access.

In [2]:
from getpass import getpass

# enter your api key
OPENAI_API_KEY = getpass("OpenAI API key: ")

Initialize the `ChatOpenAI` object. We'll set `temperature=0` to minimize randomness and make outputs repeatable.

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model='gpt-3.5-turbo'
)

Chats with the Chat-GPT model `gpt-3.5-turbo` are typically structured like so:

```
System: You are a helpful assistant.

User: Hi AI, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand string theory.

Assistant: 
```

The final `"Assistant:"` without a response is what would prompt the model to continue the comversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

```json
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand string theory."}
]
```

In LangChain there is a slightly different format. We use three *message* objects like so:

In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

The format is very similar, we're just swapper the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [6]:
res = chat(messages)
res

AIMessage(content='Sure, I can help you with that. String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not particles, but rather tiny, one-dimensional "strings" that vibrate at different frequencies. These strings are incredibly small, with a length scale of around 10^-35 meters.\n\nThe theory suggests that there are many different possible configurations of these strings, each corresponding to a different particle. For example, an electron might be a string vibrating in one way, while a photon might be a string vibrating in a different way.\n\nOne of the key features of string theory is that it requires the existence of extra dimensions beyond the three spatial dimensions we are familiar with. In fact, the theory requires a total of 10 or 11 dimensions, depending on the specific version of the theory.\n\nString theory is still a highly speculative a

In response we get another AI message object. We can print it more clearly like so:

In [7]:
print(res.content)

Sure, I can help you with that. String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not particles, but rather tiny, one-dimensional "strings" that vibrate at different frequencies. These strings are incredibly small, with a length scale of around 10^-35 meters.

The theory suggests that there are many different possible configurations of these strings, each corresponding to a different particle. For example, an electron might be a string vibrating in one way, while a photon might be a string vibrating in a different way.

One of the key features of string theory is that it requires the existence of extra dimensions beyond the three spatial dimensions we are familiar with. In fact, the theory requires a total of 10 or 11 dimensions, depending on the specific version of the theory.

String theory is still a highly speculative area of physics, and there

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [8]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it provides a framework for describing all the fundamental particles and forces of nature in a single, coherent framework. 

In the standard model of particle physics, there are four fundamental forces: gravity, electromagnetism, the strong nuclear force, and the weak nuclear force. These forces are described by different mathematical equations and are not easily reconciled with each other. 

String theory, on the other hand, proposes that all of these forces arise from the same underlying physical principles. In particular, the theory suggests that the different particles and forces are all manifestations of the same underlying strings vibrating in different ways. 

Furthermore, string theory requires the existence of extra dimensions beyond the three spatial dimensions we are familiar with. These extra dimensions could potentially provide a way to unify the different forces of nature by showin

## New Prompt Templates

Alongside what we've seen so far there are also three new prompt templates that we can use. Those are the `SystemMessagePromptTemplate`, `AIMessagePromptTemplate`, and `HumanMessagePromptTemplate`.

These are simply an extension of [Langchain's prompt templates](https://www.pinecone.io/learn/langchain-prompt-templates/) that modify the returning "prompt" to be a `SystemMessage`, `AIMessage`, or `HumanMessage` object respectively.

For now, there are not a huge number of use-cases for these objects. However, if we have something that we'd like to add to our messages, this can be helpful. For example, let's say we'd like our AI responses to always consist of no more than 50 characters.

Using the current OpenAI `gpt-3.5-turbo-0301` model, we might run into issues if passing this instruction in the first system message only.

In [10]:
chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model='gpt-3.5-turbo-0301'
)

# setup first system message
messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 characters long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    HumanMessage(content="Hi AI, how are you? What is quantum physics?")
]

Now make our first completion.

In [11]:
res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 154
I'm doing well, thank you! Quantum physics is the study of the behavior of matter and energy at a very small scale, such as atoms and subatomic particles.


Okay, seems like our AI assistant isn't so good at following either of our instructions. What if we add these instructions to the `HumanMessage` via a `HumanMessagePromptTemplate`?

In [12]:
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    '{input} Can you keep the response to no more than 100 characters '+
    '(including whitespace), and sign off with a random name like "Robot '+
    'McRobot" or "Bot Rob".'
)

# create the human message
chat_prompt = ChatPromptTemplate.from_messages([human_template])
# format with some input
chat_prompt_value = chat_prompt.format_prompt(
    input="Hi AI, how are you? What is quantum physics?"
)
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='Hi AI, how are you? What is quantum physics? Can you keep the response to no more than 100 characters (including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".', additional_kwargs={})])

Note that to use `HumanMessagePromptTemplate` as typical a prompt templates with the `.format_prompt` method, we needed to pass it through a `ChatPromptTemplate` object. This is case for all of the new chat-based prompt templates.

Using this we return a `ChatPromptValue` object. This can be formatted into a list or string like so:



In [13]:
chat_prompt_value.to_messages()

[HumanMessage(content='Hi AI, how are you? What is quantum physics? Can you keep the response to no more than 100 characters (including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".', additional_kwargs={})]

In [14]:
chat_prompt_value.to_string()

'Human: Hi AI, how are you? What is quantum physics? Can you keep the response to no more than 100 characters (including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".'

Let's see if this new approach works.

In [19]:
messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 characters long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    chat_prompt.format_prompt(
        input="Hi AI, how are you? What is quantum physics?"
    ).to_messages()[0]
]

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 99
I'm good! Quantum physics studies the behavior of matter and energy at a very small scale. -Bot Rob


This time we get pretty close, we're slightly over the character limit (by `8` characters), and we got a sign off with `- Bot Rob`.

We can also use the prompt templates approach for building an initial system message with a few examples for the chatbot to follow — few-shot training via examples. Let's see what that looks like.

In [20]:
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate
)

system_template = SystemMessagePromptTemplate.from_template(
    'You are a helpful assistant. You keep responses to no more than '
    '{character_limit} characters long (including whitespace), and sign '
    'off every message with "- {sign_off}'
)
human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response} - {sign_off}")

# create the list of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])
# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    character_limit="50", sign_off="Robot McRobot",
    input="Hi AI, how are you? What is quantum physics?",
    response="Good! It's physics of small things"
)
chat_prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot', additional_kwargs={}), HumanMessage(content='Hi AI, how are you? What is quantum physics?', additional_kwargs={}), AIMessage(content="Good! It's physics of small things - Robot McRobot", additional_kwargs={})])

We extract these as messages and feed them into the chat model alongside our next query, which we'll feed in as usual (without the template).

In [21]:
messages = chat_prompt_value.to_messages()

messages.append(
    HumanMessage(content="How small?")
)

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 41
Atoms, electrons, photons - Robot McRobot


Perfect, we seem to get a good response there, let's try a couple more.

In [22]:
# add last response
messages.append(res)

# make new query
messages.append(
    HumanMessage(content="Okay cool, so it is like 'partical physics'?")
)

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 54
Yes, it's a branch of particle physics - Robot McRobot


We went a little over here. We could begin using the previous `HumanMessagePromptTemplate` again like so:

In [23]:
from langchain import PromptTemplate

# this is a faster way of building the prompt via a PromptTemplate
human_template = HumanMessagePromptTemplate.from_template(
    '{input} Answer in less than {character_limit} characters (including whitespace).'
)
# create the human message
human_prompt = ChatPromptTemplate.from_messages([human_template])
# format with some input
human_prompt_value = human_prompt.format_prompt(
    input="Okay cool, so it is like 'partical physics'?",
    character_limit="50"
)
human_prompt_value

ChatPromptValue(messages=[HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).", additional_kwargs={})])

In [24]:
# drop the last message about partical physics so we can rewrite
messages.pop(-1)

HumanMessage(content="Okay cool, so it is like 'partical physics'?", additional_kwargs={})

In [25]:
messages.extend(human_prompt_value.to_messages())
messages

[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot', additional_kwargs={}),
 HumanMessage(content='Hi AI, how are you? What is quantum physics?', additional_kwargs={}),
 AIMessage(content="Good! It's physics of small things - Robot McRobot", additional_kwargs={}),
 HumanMessage(content='How small?', additional_kwargs={}),
 AIMessage(content='Atoms, electrons, photons - Robot McRobot', additional_kwargs={}),
 HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).", additional_kwargs={})]

Now process:

In [26]:
res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 28
Yes, similar - Robot McRobot


There we go, a good answer again!

Now, it's arguable as to whether all of the above is better than simple f-strings like:

In [27]:
_input = "Okay cool, so it is like 'partical physics'?"
character_limit = 50

human_message = HumanMessage(content=(
    f"{_input} Answer in less than {character_limit} characters "
    "(including whitespace)."
))

human_message

HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).", additional_kwargs={})

In this example, the above is far simpler. So we wouldn't necessarily recommend using prompt templates over f-strings in all scenarios. But, if you do find yourself in a scenario where they become more useful — you now know how to use them.

To finish off, let's see how the rest of the completion process can be done using the f-string formatted message `human_message`:

In [28]:
# drop the last message about partical physics so we can rewrite
messages.pop(-1)

# add f-string formatted message
messages.append(human_message)
messages

[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot', additional_kwargs={}),
 HumanMessage(content='Hi AI, how are you? What is quantum physics?', additional_kwargs={}),
 AIMessage(content="Good! It's physics of small things - Robot McRobot", additional_kwargs={}),
 HumanMessage(content='How small?', additional_kwargs={}),
 AIMessage(content='Atoms, electrons, photons - Robot McRobot', additional_kwargs={}),
 HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).", additional_kwargs={})]

In [29]:
res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 28
Yes, similar - Robot McRobot


That's it for this example exploring LangChain's new features for chat.